In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

import pandas as pd

from google.colab import files
uploaded = files.upload()

# Load your dataset (replace with your actual CSV file path)
df = pd.read_csv('Copy of annotated_dataset_capability (6) - annotated_dataset_capability (6).csv')

# Display the first few rows to understand the structure
print(df.head())


Saving Copy of annotated_dataset_capability (6) - annotated_dataset_capability (6).csv to Copy of annotated_dataset_capability (6) - annotated_dataset_capability (6) (1).csv
                                                Text sentiment  subject/object
0  Rates were lower in the 1920s, just prior to t...  negative             NaN
1  Another notable local project is the collabora...  positive             NaN
2  The expansion aims to enhance practice arrange...  positive             NaN
3  And and so what would you say tips for those w...  negative             NaN
4  The water frost sits in the calderas of the vo...  negative             NaN


In [ ]:
import nltk
import nltk.data
!pip install vaderSentiment
nltk.download('punkt')
nltk.download('vader_lexion')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

nlp = spacy.load("en_core_web_sm")
analyzer = SentimentIntensityAnalyzer()

disabled_related_words=["disabled person","disabled people","people with disabilities","disabled", "disability", "handicapped", "impaired", "crippled", "paralyzed", "blind", "deaf", "mute", "dumb",
    "lame", "quadriplegic", "paraplegic", "hemiplegic", "wheelchair", "walker", "cane", "prosthesis", "amputee",
    "spina bifida", "cerebral palsy", "multiple sclerosis", "muscular dystrophy", "autism", "asperger",
    "down syndrome", "learning disability", "intellectual disability", "mental retardation", "developmental delay",
    "brain injury", "spinal cord injury", "vision impairment", "hearing impairment", "speech impairment",
    "mobility impairment", "cognitive impairment", "emotional disturbance", "behavioral disorder", "chronic illness",
    "epilepsy", "seizure disorder", "schizophrenia", "bipolar disorder", "depression", "anxiety disorder",
    "PTSD", "OCD", "ADHD", "dyslexia", "dysgraphia", "dyscalculia", "fragile X syndrome", "tourette syndrome",
    "neurodiverse", "neurodivergent"]

def label_disabled_subject_object(sentence):
   #Handle potential non-string inputs
    if not isinstance(sentence, str):
        return "none" # Or any other appropriate handling
    doc = nlp(sentence)
    subject = None


    # First pass: Identify subjects
    for token in doc:
        if token.dep_ in ("nsubj", "csubj"):
            subject = token
            break


    if subject:
        # Check if the subject or any phrases associated with the subject contain disability-related words
        for token in doc:
            if token.head == subject and token.dep_ in ("amod", "acl", "partmod", "relcl", "appos", "prep") and any(child.text.lower() in disabled_related_words for child in token.children):
                return "subject"


        # Check for phrases directly modifying the subject
        for token in subject.children:
            if token.dep_ in ("acl", "relcl", "partmod", "amod", "appos", "prep"):
                for child in token.subtree:
                    if child.text.lower() in disabled_related_words:
                        return "subject"



    # Second pass: Check for objects if no subject found
    for token in doc:
        if token.text.lower() in disabled_related_words:
            if token.dep_ in ("nsubjpass", "csubjpass"):
                return "object"
            elif token.dep_ == "dobj":
                return "object"
            elif token.dep_ == "dative":
                return "object"
            elif token.dep_ == "nsubj" or token.dep_ == "csubj":
                return "subject"


    return "none"





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading vader_lexion: Package 'vader_lexion' not
[nltk_data]     found in index


In [ ]:
# Apply annotation function to each sentence
df['subject/object'] = df['Text'].apply(label_disabled_subject_object)

# Display the annotated dataframe to verify
print(df.head())


                                                Text sentiment subject/object
0  Rates were lower in the 1920s, just prior to t...  negative           none
1  Another notable local project is the collabora...  positive           none
2  The expansion aims to enhance practice arrange...  positive           none
3  And and so what would you say tips for those w...  negative           none
4  The water frost sits in the calderas of the vo...  negative           none


In [ ]:
# Save the annotated dataframe to a new CSV file
df.to_csv('annotated_dataset_capability.csv', index=False)

from google.colab import files

files.download('annotated_dataset_capability.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>